In [ ]:
import build_dataset

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Data Processing

In [ ]:
data = data_original[data_original['Sport'] == 'Bike'].copy()
data['mod_pow_at_threshold'] = data['a'] + data['b']*athlete_statics['threshold_hr'] +  data['c']*(60*60)*20

data = data.groupby('date').agg({'L1_Time_in_Zone':'sum'
                                ,'L2_Time_in_Zone':'sum'
                                ,'L3_Time_in_Zone':'sum'
                                ,'L4_Time_in_Zone':'sum'
                                ,'L5_Time_in_Zone':'sum'
                                ,'L6_Time_in_Zone':'sum'
                                ,'L7_Time_in_Zone':'sum'
                                ,'mod_pow_at_threshold':'max'}).reset_index()

# data['date'] = data.index
data['date'] = pd.to_datetime(data['date'])
data = data.sort_values(by=['date'])
data.index = pd.DatetimeIndex(data['date'])
missing_dates = pd.date_range(start=data.index.min(), end=data.index.max())
data = data.reindex(missing_dates, fill_value=0)
data['mod_pow_at_threshold'] = data['mod_pow_at_threshold'].replace(0,np.nan)
data['mod_pow_at_threshold'] = data['mod_pow_at_threshold'].fillna(method='ffill')
data = data.dropna()